In [21]:
!pip install --upgrade pip
!pip install cuquantum-python
!pip install nvidia-cusparse-cu12 nvidia-cublas-cu12 nvidia-cuda-runtime-cu12 custatevec_cu12
!pip install pennylane pennylane-lightning-gpu

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be fou

In [22]:
import pennylane as qml
from pennylane.optimization import max_cut
import cuquantum
import networkx as nx
import numpy as np

# グラフの定義
graph = nx.Graph()
graph.add_edge(0, 1)
graph.add_edge(1, 2)
graph.add_edge(2, 0)

# コスト関数の定義
cost_h, mixer_h = qaoa.max_cut(graph)

# デバイスの定義（cuQuantumを使用）
dev = qml.device('cuquantum.qubit', wires=range(3), shots=1024)

# QAOAの回路を定義
def qaoa_layer(gamma, alpha):
    qaoa.cost_layer(gamma, cost_h)
    qaoa.mixer_layer(alpha, mixer_h)

def circuit(params, **kwargs):
    for i in range(3):
        qml.Hadamard(wires=i)
    
    qml.layer(qaoa_layer, 2, params[0], params[1])

# コスト関数を計算する関数を定義
cost_function = qml.ExpvalCost(circuit, cost_h, dev)

# オプティマイザーの定義
opt = qml.GradientDescentOptimizer(stepsize=0.1)

# 初期パラメータの定義
params = np.array([[0.5, 0.5], [0.5, 0.5]], requires_grad=True)

# オプティマイゼーションのループ
for i in range(30):
    params = opt.step(cost_function, params)
    if (i + 1) % 5 == 0:
        print(f"Step {i + 1}: cost = {cost_function(params):.3f}")

# 最適化されたパラメータを使って回路を実行
samples = qml.sample(circuit, params, obs=qml.PauliZ)

# 結果を表示
print(f"Optimized parameters: {params}")
print(f"Samples: {samples}")
print(f"Cost: {cost_function(params):.3f}")

ModuleNotFoundError: No module named 'pennylane.optimization'